## 1. Tải dữ liệu

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2023-04-21 14:16:45--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt.1’

data.txt.1          100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2023-04-21 14:16:45 (121 MB/s) - ‘data.txt.1’ saved [93578/93578]



## 2. Thêm các thư viện cần thiết

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np

## 3. Xử lý dữ liệu

In [ ]:
corpus = data.lower().split("\n")

In [ ]:
['from fairest creatures we desire increase, <end>',
 "that thereby beauty's rose might never die, <end>",
 'but as the riper should by time decease, <end>',
 'his tender heir might bear his memory: <end>']

['from fairest creatures we desire increase, <end>',
 "that thereby beauty's rose might never die, <end>",
 'but as the riper should by time decease, <end>',
 'his tender heir might bear his memory: <end>']

In [ ]:
corpus[:4]

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:']

### 3.1. Xây dựng tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
total_words

3211

### 3.2. Tách câu

Tách từng câu thành nhiều phần có chiều dài tăng dần để làm từng điểm dữ liệu

In [ ]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
input_sequences[:10]

[[34, 417],
 [34, 417, 877],
 [34, 417, 877, 166],
 [34, 417, 877, 166, 213],
 [34, 417, 877, 166, 213, 517],
 [8, 878],
 [8, 878, 134],
 [8, 878, 134, 351],
 [8, 878, 134, 351, 102],
 [8, 878, 134, 351, 102, 156]]

In [ ]:
tokenizer.sequences_to_texts([[34, 417]])

['from fairest']

Thực chất các câu sẽ được cắt thành nhiều phần như thế này

In [ ]:
for point in input_sequences[:10]:
  print(" ".join(tokenizer.sequences_to_texts([point])))

from fairest
from fairest creatures
from fairest creatures we
from fairest creatures we desire
from fairest creatures we desire increase
that thereby
that thereby beauty's
that thereby beauty's rose
that thereby beauty's rose might
that thereby beauty's rose might never


### 3.3. Chia features, label

Thực hiện Padding

In [ ]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

Các câu được tách bên trên sẽ được padding với việc chèn các giá trị 0 vào trước câu để tạo ra các câu có chiều dài bằng nhau.

Việc **padding vào trước** vì bài toán này ta sẽ sinh từ từ phía sau nên thông tin bên phải là những giá trị khác 0.

In [ ]:
input_sequences[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,  34, 417],
       [  0,   0,   0,   0,   0,   0,   0,   0,  34, 417, 877],
       [  0,   0,   0,   0,   0,   0,   0,  34, 417, 877, 166],
       [  0,   0,   0,   0,   0,   0,  34, 417, 877, 166, 213],
       [  0,   0,   0,   0,   0,  34, 417, 877, 166, 213, 517],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   8, 878],
       [  0,   0,   0,   0,   0,   0,   0,   0,   8, 878, 134],
       [  0,   0,   0,   0,   0,   0,   0,   8, 878, 134, 351],
       [  0,   0,   0,   0,   0,   0,   8, 878, 134, 351, 102],
       [  0,   0,   0,   0,   0,   8, 878, 134, 351, 102, 156]],
      dtype=int32)

Cắt cột cuối cùng của ma trận bên trên để làm nhãn

In [ ]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
# Chuyển thành One Hot vector


In [ ]:
label[i:i+1]

array([351], dtype=int32)

In [ ]:
for i in range(10):
  print("{} ---> {}".format(" ".join(tokenizer.sequences_to_texts(predictors[i:i+1])), tokenizer.sequences_to_texts([label[i:i+1]])[0]))

from ---> fairest
from fairest ---> creatures
from fairest creatures ---> we
from fairest creatures we ---> desire
from fairest creatures we desire ---> increase
that ---> thereby
that thereby ---> beauty's
that thereby beauty's ---> rose
that thereby beauty's rose ---> might
that thereby beauty's rose might ---> never


Chuyển từng nhãn thành vector one hot với số lượng là số từ trong từ điển

In [ ]:
label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## 4. Xây dựng model

Mô hình này có nhiệm vụ học khả năng một từ hiện tại có xác suất xuất hiện sau một số lượng từ nhất định như thế nào.

Mạng bao gồm:

- 1 lớp Embedding với chiều embedding là 100
- Một lớp 1 Bidirectional với cell LSTM 150 node
- Một lớp LSTM với 100 node
- Mạng nơ ron phân loại gồm một lớp ẩn


In [ ]:
model = Sequential()

model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))

model.add(Bidirectional(LSTM(150, return_sequences = True)))

model.add(Dropout(0.2))

model.add(LSTM(100))

model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 10, 100)           321100    
                                                                 
 bidirectional_1 (Bidirectio  (None, 10, 300)          301200    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 10, 300)           0         
                                                                 
 lstm_6 (LSTM)               (None, 100)               160400    
                                                                 
 dense_6 (Dense)             (None, 1605)              162105    
                                                                 
 dense_7 (Dense)             (None, 3211)              5156866   
                                                      

Tiến hành training

In [ ]:
 history = model.fit(predictors, label, epochs=130, verbose=1)

Epoch 1/130
484/484 [==============================] - 18s 23ms/step - loss: 6.9064 - accuracy: 0.0197
Epoch 2/130
484/484 [==============================] - 5s 11ms/step - loss: 6.5003 - accuracy: 0.0208
Epoch 3/130
484/484 [==============================] - 5s 9ms/step - loss: 6.3945 - accuracy: 0.0269
Epoch 4/130
484/484 [==============================] - 5s 11ms/step - loss: 6.2762 - accuracy: 0.0307
Epoch 5/130
484/484 [==============================] - 5s 10ms/step - loss: 6.1830 - accuracy: 0.0347
Epoch 6/130
484/484 [==============================] - 5s 11ms/step - loss: 6.1019 - accuracy: 0.0379
Epoch 7/130
484/484 [==============================] - 5s 10ms/step - loss: 6.0219 - accuracy: 0.0411
Epoch 8/130
484/484 [==============================] - 5s 10ms/step - loss: 5.9467 - accuracy: 0.0424
Epoch 9/130
484/484 [==============================] - 6s 12ms/step - loss: 5.8592 - accuracy: 0.0482
Epoch 10/130
484/484 [==============================] - 5s 9ms/step - loss: 5.7547

### 5. Dự đoán 10 từ tiếp theo

Câu mồi

In [ ]:
test_seq = 'despite of wrinkles'

Từ câu mồi
- Dự đoán ra từ tiếp theo từ các từ của câu hiện tại
- Nối từ đã được dự đoán vào câu hiện tại
- Tiếp tục dùng câu hiện tại này để dự đoán
- Thực hiện đến khi chiều dài của câu đạt một giới hạn cụ thể hoặc số từ sinh ra đạt giới hạn

In [ ]:
next_words = 300

for _ in range(next_words):
  # Chuyển câu thành vector
  token_list = tokenizer.texts_to_sequences([test_seq])[0]

  # Padding câu
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

  # Dự đoán từ tiếp theo
  predicted = model.predict(token_list, verbose=0)

  output_word = ""

  predicted_id = np.argmax(predicted)



  if predicted_id in tokenizer.index_word:
    output_word = tokenizer.index_word[predicted_id]

    if output_word == '<end>':
        break
    test_seq += " " + output_word
  else:
    break

print(test_seq)


despite of wrinkles this thy golden time you must have sing dead new prove new skill dear heart was ill ' must be the wide day of thee stand green now aside seen ride deem'd worth forth out dead ill new might live free die prove hence strife groan was bad before me ill well on thee prove thee lie ' new new new another eye ill ' do ' so are false days back made back back again none lust still winter back back back back back back back converted new winter back by thence away away away away be bevel date back so canopy me might say things live back on thee might live well on thee best hence prove hence hence days lies dote grow did lie so do die none pain die spend spend loss dead ill twain deeds back behold days out new ill new pride so let me true heart blind change false on beauty might be live and hence hence words twain grow of bad hence pain live ill so let thee long thee thee good best clearer night quite clear cold new ill forth new in approve dead lie dead ' still die twain hence

Để sinh tốt hơn bạn có thể tham khảo thêm thuật toán **Beam Search**

In [ ]:
# despite of wrinkles
# this: 0.6
# that: 0.54

How old are you? As an AI language model, I do not have an age like humans do. I was created and trained by OpenAI and my knowledge is constantly being updated and improved.
How old are you? As -> an
How old are you? As an -> AI
How old are you? As an AI -> language
How old are you? As an AI language -> model

How old are you? As an AI language model, I do not have an age like humans do. I was created and trained by OpenAI and my knowledge is constantly being updated and improved. -> <end>


